In [ ]:
클릭된 광고 예측

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# 데이터
df = pd.read_csv("https://raw.githubusercontent.com/SoongMoo/data/refs/heads/main/Clicked%20Ads%20Dataset.csv")
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
X = df.select_dtypes(include=["int64", "float64"])

le = LabelEncoder()
y = le.fit_transform(df["Clicked on Ad"]) # 0, 1

# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# scaling
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

# 모델 구성
model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation='relu', 
                kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# EarlyStopping 콜백
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 학습
history = model.fit(
    X_train, y_train,
    epochs=100, 
    batch_size=32,
    validation_split=0.25,
    verbose=1,
    callbacks=[early_stopping]
)

# 평가
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"테스트 정확도: {acc:.4f}")


In [ ]:
import tensorflow as tf
Daily_Time_Spent_on_Site = 68.5
Age =  35
Area_Income = 65000
Daily_Internet_Usage = 200

input_data = [[Daily_Time_Spent_on_Site, Age, Area_Income, Daily_Internet_Usage]]
input_data = tf.constant(input_data)  

# 입력도 스케일링 필요
#input_data = np.array([[Daily_Time_Spent_on_Site, Age, Area_Income, Daily_Internet_Usage]])
#input_data = scaler.transform(input_data)

prediction = model.predict(input_data)[0][0]
print("예측 확률:", prediction)
print("광고 클릭 여부:", "클릭" if prediction >= 0.5 else "미클릭")